In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import re
from selenium import webdriver
import os, time, random
from multiprocessing import Pool
from multiprocessing import Process,Queue
# import pymssql
import sqlite3

class youxin:
    def __init__(self):
        self.conn = pymssql.connect(host='localhost:1433', user='car', password='root', database='car_info',charset='utf8', autocommit=True)  ###　这边是数据库设置，需要修改
        
        self.driver = self.__class__.getdriver(self)
    def getdriver(self):
        option = webdriver.ChromeOptions()
        #option.add_argument('--headless')
        option.add_argument('disable-infobars')
        driver = webdriver.Chrome(r'D:\chromedriver\chromedriver.exe',chrome_options=option)
        driver.maximize_window()
        return driver
    #把所有的车系页面url保存到文件中
    def LinkToExcel(self):
        content = requests.get('https://www.xin.com/nanjing/aodi/?channel=a49b117c44837d110753e751863f53').content
        soup = BeautifulSoup(content,"html.parser")
        linklist = []
        brandlist = []
        #遍历所有车型
        for dd in soup.find('ul',attrs={'class':'brand-cars clearfix'}).find_all('dd')[1:]:
            link = 'https:'+dd.a['href']
            brand = dd.text.strip()
            try:
                content1 = requests.get(link,timeout=20).content
            except:
                continue
            soup1 = BeautifulSoup(content1,'html.parser')
            #遍历所有车系
            for i in soup1.find_all('dd',attrs={'data-type':'2'}):
                link1 = 'https:'+i.a['href']
                print(brand,link1)
                #链接
                linklist.append(link1)
                #车型
                brandlist.append(brand)
        df = pd.DataFrame()
        df['brandlist'] = brandlist
        df['linklist'] = linklist
        #去重
        df.drop_duplicates(['linklist'],keep='last',inplace=True)
        #储存到excel文件上面
        df.to_excel('linklist.xls',index=False)
    #得到浮点数值
    def GetFloat(self,x):
        try:
            return re.findall(r'\d+\.\d+',x)[0]
        except:
            return ''
    def GetInt(self,x):
        try:
            return re.findall(r'\d+',x)[0]
        except:
            return ''
    def CarInfoCollection(self):
        header = {
            'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
            'Accept-Encoding':'gzip, deflate, sdch, br',
            'Accept-Language':'zh-CN,zh;q=0.8',
            'Connection':'keep-alive',
            'Cookie':'pif=%7B%22goldNum%22%3A535%2C%22silverNum%22%3A31%2C%22unAuthenNum%22%3A0%7D; RELEASE_KEY=; pif=%7B%22goldNum%22%3A2333%2C%22silverNum%22%3A157%2C%22unAuthenNum%22%3A0%7D; RELEASE_KEY=; XIN_anti_uid=19421D02-7279-B1B9-69D6-801C492B195B; uid=rBAKD1yv/pk4RheDJr+LAg==; XIN_UID_CK=d75c8aea-b53a-d2a5-1a1a-2f19cf7da94a; SEO_REF=https://www.xin.com/nanjing/zhongshun/shiji/; Hm_lvt_ae57612a280420ca44598b857c8a9712=1555037845; Hm_lpvt_ae57612a280420ca44598b857c8a9712=1555041418; XIN_LOCATION_CITY=%7B%22cityid%22%3A%221501%22%2C%22areaid%22%3A%226%22%2C%22big_areaid%22%3A%221%22%2C%22provinceid%22%3A%2215%22%2C%22cityname%22%3A%22%5Cu5357%5Cu4eac%22%2C%22ename%22%3A%22nanjing%22%2C%22shortname%22%3A%22NJ%22%2C%22service%22%3A%221%22%2C%22near%22%3A%221505%2C3001%2C101%2C1507%2C1511%2C2401%2C1502%2C1201%2C1503%2C1518%22%2C%22tianrun_code%22%3A%22025%22%2C%22zhigou%22%3A%221%22%2C%22is_visit%22%3A%221%22%2C%22longitude%22%3A%22118.7968770%22%2C%22latitude%22%3A%2232.0602550%22%2C%22city_rank%22%3A100%2C%22city_group%22%3A%223%22%2C%22is_gold_partner%22%3A%22-1%22%2C%22direct_rent_support%22%3A%221%22%2C%22salvaged_support%22%3A%221%22%2C%22isshow_c%22%3A%221%22%2C%22is_lease_back%22%3A%221%22%2C%22mortgage_service_fee%22%3A%2260000%22%2C%22is_small_pub_house%22%3A%221%22%2C%22is_wz_mortgage%22%3A%220%22%2C%22is_purchase_direct%22%3A%221%22%2C%22is_purchase_origin%22%3A%221%22%2C%22is_ms_ex%22%3A%220%22%2C%22is_ms_trans%22%3A%220%22%2C%22updatetime%22%3A%222019-03-23+22%3A43%3A33%22%2C%22region_name%22%3A%22%5Cu6d77%5Cu68e0%5Cu4e00%5Cu533a%22%2C%22huanxin_id%22%3A%2221070094-f6fa-4b27-9f46-ca906788d940%22%2C%22is_login%22%3A1%7D; NSC_ueyz_yjo_xfc_dpsf=ffffffffaf19140145525d5f4f58455e445a4a423660; session_xin=kc5vb9u4dl73ok882oghmpu2bblr22oi; NSC_my_yjo_xfc_dpsf=ffffffffaf18141e45525d5f4f58455e445a4a423660',
            'Host':'www.xin.com',
            'Referer':'https://www.xin.com/nanjing/aodi/a4l/i2/',
            'Upgrade-Insecure-Requests':'1',
            'User-Agent':'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'
        }
        carurl = []
        CARMODEL1s = []
        ModelYears = []
        BRANDs = []
        cur = self.conn.cursor()
        df = pd.read_excel('linklist.xls')
        for index,item in df.iterrows():
            link = item[1]
            print(link)
            BRAND = item[0]
            self.driver.get(link)
            soup = BeautifulSoup(self.driver.page_source,"html.parser")
            print('------------')
            try:
                pagenum = int(soup.find('div',attrs={'class':'con-page search_page_link'}).find_all('a')[-2].text)
                print(pagenum)
                for num in range(pagenum):
                    index = num+1
                    pagelink = link+'i{0}/'.format(index)
                    try:
                        content1 = requests.get(pagelink,headers=header,timeout=25).content
                    except:
                        continue
                    soup1 = BeautifulSoup(content1,"html.parser")
                    for li in soup1.find_all('li',attrs={'class':'con caritem conHeight'}):
                        carlink = 'https:'+li.find('a',attrs={'class':'aimg'})['href']#车辆链接
                        carurl.append(carlink)
                        CARMODEL1 = li['data-title']#CARMODEL1
                        CARMODEL1s.append(CARMODEL1)
                        ModelYear = re.findall(r'20\d+',CARMODEL1)[0]#ModelYear
                        ModelYears.append(ModelYear)
                        BRANDs.append(BRAND)
#                         cur.execute("insert into carurl(url) values(%s)",(carlink))
            except Exception as e:
                    print(e.args)
                    for li in soup.find_all('li',attrs={'class':'con caritem conHeight'}):
                        carlink = 'https:'+li.find('a',attrs={'class':'aimg'})['href']#车辆链接
                        carurl.append(carlink)
                        CARMODEL1 = li['data-title']#CARMODEL1
                        CARMODEL1s.append(CARMODEL1)
                        ModelYear = re.findall(r'20\d+',CARMODEL1)[0]#ModelYear
                        ModelYears.append(ModelYear)
                        BRANDs.append(BRAND)
#                         cur.execute("insert into carurl(url) values(%s)",(carlink))
        df1 = pd.DataFrame()
        df1['url'] = carurl
        df1['CARMODEL1'] = CARMODEL1s
        df1['ModelYear'] = ModelYears
        df1['BRANDs'] = BRANDs
        df1.to_csv('carurl.csv',index=False)
    def carcollection(self,item):
        cur = self.conn.cursor()
        carlink = item[0]
        n = cur.execute("select * from youxin where PageUrl=%s",(carlink))
        if n is not None:
            return ''
        CARMODEL1 = item[1]
        ModelYear = item[2]
        BRAND = item[3]
        self.driver.get(carlink)
        if '优信二手车-人机验证页' in self.driver.title:
            print('优信二手车-人机验证页')
            self.carcollection(item)
        time.sleep(1)
        try:
            self.driver.find_element_by_xpath('//*[@id="cd_m_clxx"]/div[4]').click()
        except:
            return
        jsvar = self.driver.execute_script("return jsvar;")
        soup2 = BeautifulSoup(self.driver.page_source,"html.parser")
        OEM =  soup2.find(text='车辆厂商').findNext('span').text.replace('\n','').strip()#OEM
        print(OEM)
        RETAILUC = soup2.find('p',attrs={'class':'cd_m_info_price'}).find('span',attrs={'class':'cd_m_info_jg'}).text.strip()#RETAIL-UC
        RETAILUC = self.GetFloat(RETAILUC)
        MSRPTAX = soup2.find('p',attrs={'class':'cd_m_info_price'}).find('span',attrs={'class':re.compile('cd_m_info_qgzg')}).text.strip()#MSRPTAX
        MSRPTAX = self.GetFloat(MSRPTAX)
        PLATEDATE = soup2.find('ul',attrs={'class':'cd_m_info_desc'}).find('span',attrs={'class':'cd_m_desc_key'}).text.strip()#PLATEDATE
        PLATEDATE = '-'.join(re.findall(r'\d+',PLATEDATE))+'-01'
        CKEXPDATE = soup2.find(text='年检到期').findNext('span').text#CKEXPDATE
        MILEAGE = soup2.find('div',attrs={'class':'cd_m_i_pz'}).find_all('span',attrs={'class':'cd_m_i_pz_val'})[1].text.strip()#MILEAGE
        MILEAGE = self.GetFloat(MILEAGE)
        WHEELBASE = soup2.find(text='轴距').findNext('span').text
        WHEELBASE = self.GetInt(WHEELBASE)#WHEELBASE
        FUELECONOMY = soup2.find('span',text=re.compile('综合油耗')).findNext('span').text#FUELECONOMY
        FUELECONOMY = self.GetInt(FUELECONOMY)#FUELECONOMY
        EMISSION = soup2.find('span',text=re.compile('排放标准')).findNext('span').text
        try:
            OFFERDATE = jsvar['first_publishtime']
        except:
            OFFERDATE = ''
        IFBUSINESS = soup2.find('span',text=re.compile('使用性质')).findNext('span').text.replace('\n','').strip()
        IFINSURANCE = soup2.find('div',attrs={'class':'cd_m_pop_clxx_right'}).find_all('img')[0]
        IFINSURANCE = re.findall(r'\d+',str(IFINSURANCE))[0]
        DRIVETYPE = soup2.find('span',text=re.compile('驱动方式')).findNext('span').text.replace('\n','').strip()
        SMT = soup2.find('span',text=re.compile('保养情况')).findNext('span').text.replace('\n','').strip()
        FUEL = soup2.find('span',text=re.compile('燃油标号')).findNext('span').text.replace('\n','').strip()
        SALESDATE = soup2.find('span',text=re.compile('上市时间')).findNext('span').text.replace('\n','').strip()
        try:
            city = jsvar['car_city']['car_cityname']
        except:
            city = ''
        try:
            city2 = jsvar['cityname']
        except:
            city2 = ''
        try:
            TRANSFERTIMES = jsvar['saletime']
        except:
            TRANSFERTIMES = ''
        try:
            DEALER = jsvar['dealername']
        except:
            DEALER = ''
        try:
            carid = jsvar['carid']
        except:
            carid = ''
        sql = "insert into youxin(BRAND,OEM,CARMODEL1,ModelYear,MSRPTAX,RETAILUC,PLATEDATE,CKEXPDATE,MILEAGE,WHEELBASE,FUELECONOMY,EMISSION,OFFERDATE,DATADATE,DEALER,CITY,IFBUSINESS,IFINSURANCE,DRIVETYPE,TRANSFERTIMES,4SMT,FUEL,SALESDATE,CarID,CITY2,PageUrl) values('%s','%s','%s','%s','%s','%s','%s','%s','%s','%s','%s','%s','%s','%s','%s','%s','%s','%s','%s','%s','%s','%s','%s','%s','%s','%s')"
        params = (BRAND,OEM,CARMODEL1,str(ModelYear).replace("'","\""),MSRPTAX,RETAILUC,PLATEDATE,CKEXPDATE,MILEAGE.replace("'","\""),WHEELBASE,FUELECONOMY,EMISSION,OFFERDATE,time.strftime('%Y%m%d'),DEALER,city,IFBUSINESS,IFINSURANCE,DRIVETYPE,TRANSFERTIMES,SMT,FUEL,SALESDATE,carid,city2,carlink)
        print(params)
        cur.execute(sql,params)
def write(q):
	df = pd.read_csv("carurl.csv")
	df.drop_duplicates('url',inplace=True)
	for index,item in df.iterrows():
		q.put(item)
		#time.sleep(random.random())
def read(q):
	print('读取进程------')
	c = youxin()
	while(True):
		if not q.empty():
			item = q.get(True)
			c.carcollection(item)
		else:
			break

if __name__ == '__main__':
    c = youxin()
    #如果不存在一级页面的url，则更新
    if not os.path.exists('linklist.xls'):
        c.LinkToExcel()
    #c.CarInfoCollection()
    #del c
    q = Queue()
    pw = Process(target=write,args=(q,))
    pr1 = Process(target=read,args=(q,))
    pr2 = Process(target=read,args=(q,))
    pr3 = Process(target=read,args=(q,))
    pw.start()

    pr1.start()
    pr2.start()
    pr3.start()
    pw.join()
    pr1.join()
    pr2.join()
    pr3.join()



    


NameError: name 'pymssql' is not defined

In [3]:
!conda install pymssql

Solving environment: - ^C
failed

CondaError: KeyboardInterrupt



In [9]:
sql = "insert into youxin(BRAND,OEM,CARMODEL1,ModelYear,MSRPTAX,RETAILUC,PLATEDATE,CKEXPDATE,MILEAGE,WHEELBASE,FUELECONOMY,EMISSION,OFFERDATE,DATADATE,DEALER,CITY,IFBUSINESS,IFINSURANCE,DRIVETYPE,TRANSFERTIMES,4SMT,FUEL,SALESDATE,CarID,CITY2,PageUrl) values('%s','%s','%s','%s','%s','%s','%s','%s','%s','%s','%s','%s','%s','%s','%s','%s','%s','%s','%s','%s','%s','%s','%s','%s','%s','%s')"
params = ('奥迪', '一汽-大众奥迪', '奥迪 A3两厢 2014款 1.4T 自动 35TFSI进取型', '2014', '13.28', '12.41', '2014-09-01', '2020-09-01', '5.1', '2629', '5', '国5', '2019-04-04T11:44:44Z', '20190423', '石家庄瑞诚安丽精品车行（2队）', '石家庄', '非营运', '201512', '前驱', '0', '无', '97(京95)', '2014-03-20', '57288770', '南京', 'https://www.xin.com/40d0jyywy9/che57288770.html?cityid=1501')
params


26

In [12]:
a = ('%s','%s','%s','%s','%s','%s','%s','%s','%s','%s','%s','%s','%s','%s','%s','%s','%s','%s','%s','%s','%s','%s','%s','%s','%s','%s')

26